In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from IPython.display import display, clear_output
import imageio.v3 as iio

# Reload modules to ensure we use the new data_loader
import importlib
import src.config
import src.transform
import src.data_loader
import src.metrics
import src.codec
import src.experiments



importlib.reload(src.config)
importlib.reload(src.transform)
importlib.reload(src.data_loader)
importlib.reload(src.metrics)
importlib.reload(src.codec)
importlib.reload(src.experiments)

from src.config import VSTConfig
from src.metrics import QualityMetrics
from src.data_loader import SyntheticGenerator, ImageLoader
from src.codec import BPGCodec
from src.experiments import RateDistortionRunner
from src.transform import VarianceStabilizer

In [38]:
class OOPAnalysisApp:
    def __init__(self, 
                 path_noised: str, 
                 path_original: str, 
                 bpg_path: str = 'bpg-0.9.8-win64'):
        
        self.bpg_path = bpg_path
        self.codec = BPGCodec(bpg_path)
        self.runner = RateDistortionRunner(self.codec)
        
        self.cached_gen = None
        self.last_noise = -1
        self.default_noised = path_noised
        self.default_original = path_original
        
        self._init_ui()
        
    def _init_ui(self):
        s = {'description_width': 'initial'}
        
        # Data Selection
        self.w_source = widgets.Dropdown(options=[('Generator', 'gen'), ('Files', 'file')], value='gen', description='Source:', style=s)
        self.w_noise = widgets.FloatSlider(value=0.05, min=0.01, max=0.5, step=0.01, description='Speckle Sigma:', style=s)
        self.w_path_noised = widgets.Text(value=self.default_noised, description='Noised:', layout=widgets.Layout(width='200px'))
        self.w_path_original = widgets.Text(value=self.default_original, description='Ref:', layout=widgets.Layout(width='200px'))
        
        # VST Params
        self.w_a = widgets.FloatSlider(value=8.39, min=1.0, max=20.0, description='a:', style=s)
        self.w_b = widgets.FloatSlider(value=1.2, min=1.05, max=5.0, description='b:', style=s)
        
        # Range (UPDATED WIDTHS)
        self.w_q_start = widgets.IntText(value=20, description='Q Start:', style=s, layout=widgets.Layout(width='120px'))
        self.w_q_end = widgets.IntText(value=51, description='Q End:', style=s, layout=widgets.Layout(width='120px'))
        self.w_q_step = widgets.IntText(value=1, description='Step:', style=s, layout=widgets.Layout(width='120px'))
        
        # Checkbox for Saving
        self.w_save = widgets.Checkbox(value=True, description='Save Result Images')
        
        self.btn_run = widgets.Button(description='Find OOP', button_style='danger', icon='crosshairs')
        self.btn_run.on_click(self.run_analysis)
        
        self.prog_bar = widgets.IntProgress(value=0, min=0, max=100, layout=widgets.Layout(width='100%'))
        self.prog_bar.layout.visibility = 'hidden'
        
        self.out_res = widgets.Output()
        
        # Layout
        src_row = widgets.HBox([self.w_source, self.w_noise, self.w_path_noised, self.w_path_original])
        param_row = widgets.HBox([self.w_a, self.w_b, self.w_q_start, self.w_q_end, self.w_q_step, self.w_save])
        
        def on_src(c):
            is_gen = (c['new'] == 'gen')
            self.w_noise.layout.display = 'flex' if is_gen else 'none'
            self.w_path_noised.layout.display = 'none' if is_gen else 'flex'
            self.w_path_original.layout.display = 'none' if is_gen else 'flex'
        
        self.w_source.observe(on_src, names='value')
        on_src({'new': 'gen'})
        
        display(widgets.VBox([
            widgets.HTML("<h3>OOP Finder & Visual Saver</h3>"),
            src_row, param_row, 
            self.btn_run, self.prog_bar, 
            self.out_res
        ]))

    def get_data(self):
        if self.w_source.value == 'gen':
            val = self.w_noise.value
            if self.cached_gen is None or val != self.last_noise:
                self.cached_gen = SyntheticGenerator.get_data(val)
                self.last_noise = val
            return self.cached_gen, '.png' # Генератор повертає PNG за замовчуванням
        else:
            try:
                path_n = self.w_path_noised.value
                i_n = ImageLoader.load_file(path_n)
                i_o = ImageLoader.load_file(self.w_path_original.value)
                # Визначаємо розширення файлу (.png, .tiff, etc.)
                _, ext = os.path.splitext(path_n)
                return (i_o, i_n), ext
            except: return None, None

    def _save_visual_result(self, img_float, filename):
        """Helper to save float image as uint8 image."""
        # Clip to valid range and convert to uint8
        img_uint8 = np.clip(img_float, 0, 255).astype(np.uint8)
        
        # Save to results folder
        save_dir = "results"
        os.makedirs(save_dir, exist_ok=True)
        full_path = os.path.join(save_dir, filename)
        
        iio.imwrite(full_path, img_uint8)
        return full_path

    def run_analysis(self, b):
        self.btn_run.disabled = True
        self.out_res.clear_output()
        self.prog_bar.layout.visibility = 'visible'
        
        try:
            data_pack, file_ext = self.get_data()
            if data_pack is None:
                with self.out_res: print("Error: Reference image needed.")
                return
            
            img_ref, img_noised = data_pack

            q_rng = list(range(self.w_q_start.value, self.w_q_end.value + 1, self.w_q_step.value))
            vst_cfg = VSTConfig(a=self.w_a.value, b=self.w_b.value)
            
            # 1. Run Analysis Curves
            self.prog_bar.description = "VST Mode..."
            res_vst = self.runner.run_curve(img_ref, img_noised, vst_cfg, q_rng, use_vst=True, progress_callback=lambda c, t: setattr(self.prog_bar, 'value', c))
            
            self.prog_bar.description = "Linear Mode..."
            res_lin = self.runner.run_curve(img_ref, img_noised, vst_cfg, q_rng, use_vst=False, progress_callback=lambda c, t: setattr(self.prog_bar, 'value', c))
            
            # 2. Identify OOPs
            def find_oop(res):
                idx = np.argmax(res['psnr'])
                return {k: res[k][idx] for k in res.keys()}, idx
            
            oop_vst, _ = find_oop(res_vst)
            oop_lin, _ = find_oop(res_lin)
            
            # 3. SAVE LOGIC (Visual Results)
            save_msg = ""
            if self.w_save.value:
                # -- A. Save VST Result --
                # Ми повинні повторити процес для найкращого Q, щоб отримати картинку
                vst = VarianceStabilizer(vst_cfg)
                img_log = vst.forward(img_noised)
                
                # Стискаємо і отримуємо відновлене (але ще в Log домені) + розмір BPG
                img_log_dec, size_bytes_vst, _ = self.codec.compress_decompress(img_log, oop_vst['q'])
                
                # Повертаємо в лінійний простір (те, що ми хочемо бачити)
                img_final_vst = vst.inverse(img_log_dec)
                
                # Формуємо ім'я: додаємо розмір BPG, щоб знати, скільки він займав
                size_kb_vst = size_bytes_vst / 1024.0
                fname_vst = f"OOP_VST_Q{oop_vst['q']}_{size_kb_vst:.1f}KB{file_ext}"
                path_vst = self._save_visual_result(img_final_vst, fname_vst)
                
                # -- B. Save Linear Result --
                img_lin_dec, size_bytes_lin, _ = self.codec.compress_decompress(img_noised, oop_lin['q'])
                size_kb_lin = size_bytes_lin / 1024.0
                fname_lin = f"OOP_Linear_Q{oop_lin['q']}_{size_kb_lin:.1f}KB{file_ext}"
                path_lin = self._save_visual_result(img_lin_dec, fname_lin)
                
                save_msg = f"""
                <div style='border:1px solid green; padding:10px; margin-top:10px'>
                    <b>✅ Visual Results Saved (in 'results/'):</b><br>
                    Format: {file_ext} (Visual copy of the compressed data)<br>
                    <ul>
                        <li><b>VST Method:</b> {fname_vst} (derived from {size_kb_vst:.1f} KB .bpg)</li>
                        <li><b>Linear Method:</b> {fname_lin} (derived from {size_kb_lin:.1f} KB .bpg)</li>
                    </ul>
                </div>
                """

            # 4. Display Results
            df = pd.DataFrame([
                {'Method': 'Standard', 'Q(OOP)': oop_lin['q'], 'PSNR': f"{oop_lin['psnr']:.2f}", 'HVS-M': f"{oop_lin['psnr_hvsm']:.2f}", 'CR': f"{oop_lin['cr']:.1f}"},
                {'Method': 'Proposed (VST)', 'Q(OOP)': oop_vst['q'], 'PSNR': f"{oop_vst['psnr']:.2f}", 'HVS-M': f"{oop_vst['psnr_hvsm']:.2f}", 'CR': f"{oop_vst['cr']:.1f}"}
            ])
            
            with self.out_res:
                display(df)
                if save_msg: display(widgets.HTML(save_msg))
                
                # Plotting
                fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                
                # PSNR
                ax1 = axes[0]
                ax1.plot(res_lin['q'], res_lin['psnr'], 'b--', label='Linear')
                ax1.plot(res_vst['q'], res_vst['psnr'], 'r-', label='VST')
                ax1.scatter(oop_lin['q'], oop_lin['psnr'], s=150, c='blue', marker='*', label='OOP Linear')
                ax1.scatter(oop_vst['q'], oop_vst['psnr'], s=150, c='red', marker='*', label='OOP VST')
                ax1.set_title("PSNR vs Q"); ax1.set_xlabel("Q"); ax1.grid(True, alpha=0.3); ax1.legend()
                
                # HVS-M
                ax2 = axes[1]
                ax2.plot(res_lin['q'], res_lin['psnr_hvsm'], 'b--', label='Linear')
                ax2.plot(res_vst['q'], res_vst['psnr_hvsm'], 'r-', label='VST')
                ax2.scatter(oop_lin['q'], oop_lin['psnr_hvsm'], s=150, c='blue', marker='*')
                ax2.scatter(oop_vst['q'], oop_vst['psnr_hvsm'], s=150, c='red', marker='*')
                ax2.set_title("HVS-M vs Q"); ax2.set_xlabel("Q"); ax2.grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()

        except Exception as e:
            with self.out_res: print(f"Error: {e}")
            import traceback
            traceback.print_exc()
        finally:
            self.btn_run.disabled = False
            self.prog_bar.layout.visibility = 'hidden'

In [ ]:
app_oop = OOPAnalysisApp(
    path_noised='data/NOISED_2.png',
    path_original='data/ORIGINAL_2.png',
    bpg_path='bpg-0.9.8-win64'
)

In [40]:
app_oop = OOPAnalysisApp(
    path_noised='data/NOISED.tiff', 
    path_original='data/ORIGINAL.tiff',
    bpg_path='bpg-0.9.8-win64'
)